Jupyter notebook for quickly testing accuracy/F1-score

imports necessary packages

In [6]:
import os
import numpy as np
import cv2
from matplotlib import pyplot as plt

In [7]:
from random import shuffle 
from tqdm import tqdm

In [8]:
import tflearn
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression

import tensorflow as tf

Instructions for updating:
Use the retry module or similar alternatives.
hdf5 is not supported on this machine (please install/reinstall h5py for optimal experience)


Define constants and directories

In [18]:
LR = 1e-3
IMG_DIR = '/home/pallab/gestures-cnn/images/qtest/'
IMG_SIZE = 100
MODEL_DIR = '/home/pallab/gestures-cnn/tfmodels/'
MODEL_NAME = 'gesture-{}-{}.model'.format(LR, 'commit') 

In [19]:
def process_test_data():
    testing_data = []
    for img in tqdm(os.listdir(IMG_DIR)):
        path = os.path.join(IMG_DIR,img)
        img_num = img[0]
        img = cv2.imread(path,cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, (IMG_SIZE,IMG_SIZE))
        testing_data.append([np.array(img), img_num])
        
    shuffle(testing_data)
    np.save('test_data.npy', testing_data)
    return testing_data

In [20]:
tf.reset_default_graph()
convnet = input_data(shape=[None, IMG_SIZE, IMG_SIZE, 1], name='input')

# 7 Layered CNN Architecture with 5 Convolutional and 2 FC Layer with Dropout and Max Pooling

convnet = conv_2d(convnet, 32, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)
convnet = dropout(convnet, 0.8)
print(convnet.shape)

convnet = conv_2d(convnet, 64, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)
convnet = dropout(convnet, 0.8)
print(convnet.shape)

convnet = conv_2d(convnet, 128, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)
convnet = dropout(convnet, 0.8)
print(convnet.shape)

convnet = conv_2d(convnet, 256, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)
convnet = dropout(convnet, 0.8)
print(convnet.shape)

convnet = conv_2d(convnet, 512, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)
convnet = dropout(convnet, 0.8)
print(convnet.shape)

convnet = fully_connected(convnet, 1024, activation='relu')
convnet = dropout(convnet, 0.8)

convnet = fully_connected(convnet, 4, activation='softmax')
convnet = regression(convnet, optimizer='adam', learning_rate=LR, loss='categorical_crossentropy', name='targets')

(?, 20, 20, 32)
(?, 4, 4, 64)
(?, 1, 1, 128)
(?, 1, 1, 256)
(?, 1, 1, 512)


In [21]:
model = tflearn.DNN(convnet)
model.load(MODEL_DIR + MODEL_NAME)

INFO:tensorflow:Restoring parameters from /home/pallab/gestures-cnn/tfmodels/gesture-0.001-commit.model


In [22]:
try:
    test_data = np.load(IMG_DIR + 'test_data.npy')
except (FileNotFoundError):
    test_data = process_test_data()

100%|██████████| 1/1 [00:01<00:00,  1.69s/it]


In [27]:
fig = plt.figure()
mark = {"i": 1, "v": 2, "f": 3, "t": 4}
labelss = []
predics = []
row, col = 1, 1

<Figure size 432x288 with 0 Axes>

In [65]:
str_label = ""
print("len =", len(test_data))
# for num, data in enumerate(test_data):
#     img_num = data[1]
#     img_data = data[0]
#     labelss.append(mark[img_num])
#     y = fig.add_subplot(row, col, num+1)
#     orig = img_data
#     data = img_data.reshape(IMG_SIZE,IMG_SIZE,1)
#     model_out = (model.predict([data])[0]).round()
#     if np.array_equal((model_out),np.array([1.,0.,0.,0.])):
#         str_label = 'index'
#         predics.append(1)
#     elif np.array_equal((model_out),np.array([0.,1.,0.,0.]) ): 
#         str_label = 'vshape'
#         predics.append(2)
#     elif np.array_equal((model_out) , np.array([0.,0.,1.,0.])): 
#         str_label = 'fist'
#         predics.append(3)
#     elif np.array_equal((model_out) , np.array([0.,0.,0.,1.])): 
#         str_label = 'thumb'
#         predics.append(4)
    
#     y.imshow(orig,cmap='gray')
#     plt.title(str_label)
#     y.axes.get_xaxis().set_visible(False)
#     y.axes.get_yaxis().set_visible(False)
num, data = test_data[0]
label = data
print("label =", label)
rep = num.reshape(IMG_SIZE,IMG_SIZE,1)
model_out = model.predict([rep])[0]

print(model_out)

len = 1
label = v
[0.58416516 0.29919818 0.05289126 0.06374542]


In [64]:
plt.tight_layout()
plt.show()

<Figure size 432x288 with 0 Axes>

In [40]:
from sklearn.metrics import classification_report
print(classification_report(labelss, predics))

             precision    recall  f1-score   support

          1       0.00      0.00      0.00         0
          2       0.00      0.00      0.00         5

avg / total       0.00      0.00      0.00         5



/home/pallab/cvp/lib/python3.5/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/pallab/cvp/lib/python3.5/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [41]:
print(labelss, predics)
print(str_label)

[2, 2, 2, 2, 2] [1, 1, 1, 1, 1]
index
